In [ ]:
!apt-get install yasm

Install ffmpeg and libavcodec-dev (plus related stuff) from
https://launchpad.net/~mc3man/+archive/ubuntu/trusty-media

Install OpenCV using the instructions from 
http://www.pyimagesearch.com/2015/06/22/install-opencv-3-0-and-python-2-7-on-ubuntu/

In my case I had a build error related to missing CUDA libs, so the build command I used was

    cmake -D CMAKE_BUILD_TYPE=RELEASE \
            -D CMAKE_INSTALL_PREFIX=/usr/local \
            -D INSTALL_C_EXAMPLES=OFF \
            -D INSTALL_PYTHON_EXAMPLES=ON \
            -D "OPENCV_EXTRA_MODULES_PATH=../../opencv_contrib/modules" \
            -D BUILD_EXAMPLES=ON \
            -D WITH_CUDA=OFF ..



In [ ]:
!pip install pytube

In [1]:
!mkdir videos
!mkdir imageseries

mkdir: cannot create directory ‘videos’: File exists


In [ ]:
from pytube import YouTube

# not necessary, just for demo purposes.
from pprint import pprint

yt = YouTube("https://www.youtube.com/watch?v=yim23IFJOyQ")

yt.set_filename('Chimpanzees1')
# Once set, you can see all the codec and quality options YouTube has made
# available for the perticular video by printing videos.

# pprint(yt.get_videos())

In [ ]:
video = yt.get('mp4', '360p')

In [ ]:
# In this case, we'll need to specify both the codec (mp4) and resolution
# (either 360p or 720p).

# Okay, let's download it!
video = yt.get('mp4', '360p')
video.download('./videos/')


In [ ]:
# Now let's try to read this using our fresh opencv build..

In [2]:
import cv2
import h5py


In [3]:
from IPython.display import HTML, FileLink, display
from base64 import b64encode
def link_video(path):
    src = FileLink(path)
    video_tag = '<video controls alt="test" src="%s">' % (path)
    
    display(HTML(data=video_tag))


In [4]:
vc = cv2.VideoCapture('videos/Chimpanzees1.mp4')
frames = 0
shape = None
dtype = None
while (True):
    ret, img = vc.read()
    # img should be an numpy ndarray of shape (width, heigth, channels)
    if (not ret):
        break
    if (shape is None):
        shape = img.shape
    if (dtype is None):
        dtype = img.dtype
    frames+=1
    
print "Could read %d frames of shape %r and dtype %r" % (frames, shape, dtype)

Could read 1490 frames of shape (360, 640, 3) and dtype dtype('uint8')


In [5]:
import h5py

h5f = h5py.File('videos/Chimpanzees1.hdf5', 'w')
h5g = h5f.create_group('input')

In [6]:
import numpy as np

In [7]:
ishape = tuple([ frames ] + list(shape))
ichunkshape = tuple([ 1 ] + list(shape))
imaxshape = tuple([ None ] + list(shape))

In [10]:
from numpy.random import permutation


In [8]:
inframes = h5g.create_dataset('raw_hsl_frames', 
                             shape=ishape, dtype=np.float32, 
                             chunks=ichunkshape, maxshape=imaxshape,
                             fletcher32=True, fillvalue=0)


In [11]:
permuted_indices = permutation(frames)

In [12]:
print shape

(360, 640, 3)


In [13]:
vc = cv2.VideoCapture('videos/Chimpanzees1.mp4')
frames = 0
shape = None
dtype = None
while (True):
    ret, img = vc.read()
    if (not ret):
        break
    hsvimg = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #print hsvimg.dtype
    
    hsvimgf = hsvimg.astype(np.float32) / 255.0
    # img should be an numpy ndarray of shape (width, heigth, channels)
    inframes[permuted_indices[frames],:,:,:] = hsvimgf
    
    hsvimg2 = (inframes[permuted_indices[frames],:,:,:] * 255.0).astype(np.uint8)
    
    #print np.sum(hsvimg-hsvimg2)
    #bgrimg = cv2.cvtColor(hsvimg2, cv2.COLOR_HSV2BGR)
    #print np.sum(img-bgrimg)
    
    #cv2.imwrite("videos/recoded1_%05d.png" % (frames), bgrimg)
    
    frames+=1
    if (frames % 50==0):
        print "Frame #%d" % (frames)
print "Read %d frames" % (frames)


Frame #50
Frame #100
Frame #150
Frame #200
Frame #250
Frame #300
Frame #350
Frame #400
Frame #450
Frame #500
Frame #550
Frame #600
Frame #650
Frame #700
Frame #750
Frame #800
Frame #850
Frame #900
Frame #950
Frame #1000
Frame #1050
Frame #1100
Frame #1150
Frame #1200
Frame #1250
Frame #1300
Frame #1350
Frame #1400
Frame #1450
Read 1490 frames


In [ ]:
inframes.shape

In [14]:
ls -hs videos/Chimpanzees1.hdf5

3,9G videos/Chimpanzees1.hdf5


In [15]:
import numpy as np

In [16]:
rm -rf videos/Chimpanzee1RegionTest_*

In [17]:
!rm videos/imageseries*.png
!ffmpeg -i videos/Chimpanzees1.mp4 -r 6 -f image2 imageseries/Chimpanzees1_%05d.png

ffmpeg version N-77455-g4707497 Copyright (c) 2000-2015 the FFmpeg developers
  built with gcc 4.8 (Ubuntu 4.8.4-2ubuntu1~14.04)
  configuration: --extra-libs=-ldl --prefix=/opt/ffmpeg --mandir=/usr/share/man --enable-avresample --disable-debug --enable-nonfree --enable-gpl --enable-version3 --enable-libopencore-amrnb --enable-libopencore-amrwb --disable-decoder=amrnb --disable-decoder=amrwb --enable-libpulse --enable-libdcadec --enable-libfreetype --enable-libx264 --enable-libx265 --enable-libfdk-aac --enable-libvorbis --enable-libmp3lame --enable-libopus --enable-libvpx --enable-libspeex --enable-libass --enable-avisynth --enable-libsoxr --enable-libxvid --enable-libvo-aacenc --enable-libvidstab
  libavutil      55. 11.100 / 55. 11.100
  libavcodec     57. 20.100 / 57. 20.100
  libavformat    57. 20.100 / 57. 20.100
  libavdevice    57.  0.100 / 57.  0.100
  libavfilter     6. 21.101 /  6. 21.101
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  0.100 /  4.  0.100
  libs

In [19]:
!rm videos/*.png
!ls videos

Chimpanzees1			    Chimpanzees1.webm
Chimpanzees1.avi		    Chimpanzees2.mp4
Chimpanzees1.hdf5		    frames
Chimpanzees1.mp4		    testX.avi
Chimpanzees1_recoded.mp4	    testX.mp4
Chimpanzees1_reconstruction2.mp4    TestX.mp4
Chimpanzees1_reconstruction3.mp4    writer_test4.mp4
Chimpanzees1_reconstruction4.mp4    writer_test5.mp4
Chimpanzees1_reconstruction5.mp4    writer_test6.mp4
Chimpanzees1RegionTest_recoded.mp4  writer_test7.mp4


In [20]:
!rm videos/Chimpanzees1_recoded.mp4
!ffmpeg -i imageseries/Chimpanzees1_%05d.png -b 512 -c:v libx264 -b:v 400k -r 10 -preset medium videos/Chimpanzees1_recoded.mp4

ffmpeg version N-77455-g4707497 Copyright (c) 2000-2015 the FFmpeg developers
  built with gcc 4.8 (Ubuntu 4.8.4-2ubuntu1~14.04)
  configuration: --extra-libs=-ldl --prefix=/opt/ffmpeg --mandir=/usr/share/man --enable-avresample --disable-debug --enable-nonfree --enable-gpl --enable-version3 --enable-libopencore-amrnb --enable-libopencore-amrwb --disable-decoder=amrnb --disable-decoder=amrwb --enable-libpulse --enable-libdcadec --enable-libfreetype --enable-libx264 --enable-libx265 --enable-libfdk-aac --enable-libvorbis --enable-libmp3lame --enable-libopus --enable-libvpx --enable-libspeex --enable-libass --enable-avisynth --enable-libsoxr --enable-libxvid --enable-libvo-aacenc --enable-libvidstab
  libavutil      55. 11.100 / 55. 11.100
  libavcodec     57. 20.100 / 57. 20.100
  libavformat    57. 20.100 / 57. 20.100
  libavdevice    57.  0.100 / 57.  0.100
  libavfilter     6. 21.101 /  6. 21.101
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  0.100 /  4.  0.100
  libs

In [21]:
!ls -lahs videos/Chimpanzees1*.mp4

5,3M -rw-rw-r-- 1 kailondenberg kailondenberg 5,3M Jan 12 01:23 videos/Chimpanzees1.mp4
840K -rw-rw-r-- 1 kailondenberg kailondenberg 840K Jan 28 16:59 videos/Chimpanzees1_recoded.mp4
2,8M -rw-rw-r-- 1 kailondenberg kailondenberg 2,8M Jan 28 11:58 videos/Chimpanzees1_reconstruction2.mp4
3,4M -rw-rw-r-- 1 kailondenberg kailondenberg 3,4M Jan 28 13:37 videos/Chimpanzees1_reconstruction3.mp4
3,9M -rw-rw-r-- 1 kailondenberg kailondenberg 3,9M Jan 28 13:54 videos/Chimpanzees1_reconstruction4.mp4
3,8M -rw-rw-r-- 1 kailondenberg kailondenberg 3,8M Jan 28 15:17 videos/Chimpanzees1_reconstruction5.mp4
760K -rw-rw-r-- 1 kailondenberg kailondenberg 758K Jan 25 22:17 videos/Chimpanzees1RegionTest_recoded.mp4


In [22]:
link_video('videos/Chimpanzees1_recoded.mp4')

In [23]:
import os
import h5py
from glob import glob



In [24]:
!cat ~/.keras/keras.json

{"epsilon": 1e-07, "floatx": "float32", "backend": "tensorflow"}


Note:

Read http://arxiv.org/pdf/1402.3337.pdf and use the ThresholdedReLu activation from http://keras.io/layers/advanced_activations/


In [25]:

import numpy as np
np.random.seed(1337)  # for reproducibility


from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape, Permute
from keras.layers.advanced_activations import ThresholdedReLU
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution2D, ZeroPadding2D
from keras.datasets import imdb
from keras.optimizers import SGD
from keras.regularizers import l1,l2,l1l2
from keras import backend as K

Using TensorFlow backend.


In [26]:
import keras.layers.convolutional as kconv

In [27]:
print 'Build model...X'

num_feature_detectors=30
tile_size=16
receptive_field_size=24
model = Sequential()
model.add(Dropout(0.5, input_shape=ichunkshape[1:]))

input_features = Convolution2D( name='encode_tiled_features', 
                        nb_filter=num_feature_detectors, nb_row=receptive_field_size,nb_col=receptive_field_size, 
                        subsample=(tile_size,tile_size), 
                        border_mode='valid', trainable=True,
                        dim_ordering='tf' )


print np.prod(ichunkshape[1:])
model.add(input_features)

print input_features.output_shape 

model.add(Activation('relu'))

decoder = Convolution2D( name='decode_tiled_features', 
                        nb_filter=tile_size*tile_size*3, nb_row=1,nb_col=1, subsample=(1,1),
                        border_mode='same', trainable=True,
                        dim_ordering='tf')

model.add(decoder)
model.add(Reshape(tuple(list(decoder.output_shape[1:3]) + [tile_size, tile_size, 3])))
print model.output_shape
model.add(Permute((1,3,2,4,5)))

print model.output_shape
model.add(Reshape((model.output_shape[1]*model.output_shape[2],model.output_shape[3]*model.output_shape[4],model.output_shape[5])))

model.add(ZeroPadding2D(((ichunkshape[1]-model.output_shape[1])/2, (ichunkshape[2]-model.output_shape[2])/2), 
                        dim_ordering='tf'))

print model.output_shape
def mean_abs_error_all(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis=None)



print model.output_shape

Build model...X
691200
(None, 22, 39, 30)
(None, 22, 39, 16, 16, 3)
(None, 22, 16, 39, 16, 3)
(None, 360, 640, 3)
(None, 360, 640, 3)


In [ ]:
print np.mean(w1)
print np.mean(w2)

In [182]:
opt = SGD(lr=0.02, momentum=0.6, decay=0.002, nesterov=True)

In [188]:
model.compile(optimizer='adam', loss=mean_abs_error_all, class_mode='binary')

In [192]:
def data_generator(batchsize):
    for i in range(10):
        for j in permutation(1300):
            bi = inframes[j:(j+batchsize),:,:,:]
            yield bi,  bi
    
for i in range(1):
    print "Over-Epoch %i" % (i)
    model.fit_generator(data_generator(8), validation_data=(inframes[1400:1430], inframes[1400:1430]),
              show_accuracy=False, nb_epoch=8, samples_per_epoch=300, nb_worker=1)

Over-Epoch 0
Epoch 1/8
304/300 [==============================] - 25s - loss: 0.0684 - val_loss: 0.0643
Epoch 2/8
304/300 [==============================] - 25s - loss: 0.0675 - val_loss: 0.0637
Epoch 3/8
304/300 [==============================] - 25s - loss: 0.0667 - val_loss: 0.0632
Epoch 4/8
304/300 [==============================] - 25s - loss: 0.0668 - val_loss: 0.0634
Epoch 5/8
304/300 [==============================] - 24s - loss: 0.0669 - val_loss: 0.0641
Epoch 6/8
304/300 [==============================] - 24s - loss: 0.0666 - val_loss: 0.0706
Epoch 7/8
304/300 [==============================] - 24s - loss: 0.0679 - val_loss: 0.0631
Epoch 8/8
304/300 [==============================] - 24s - loss: 0.0671 - val_loss: 0.0645


In [ ]:
for bi, ba in data_generator(2):
    print bi.shape
    print ba.shape

In [ ]:
predictions = model.predict(inframes[1:4], batch_size=4)

In [ ]:
reverse_permute = np.array(permuted_indices, dtype=np.int32)
for i in range(frames):
    reverse_permute[permuted_indices[i]]=i


In [197]:
!rm videos/reco*.png

rm: cannot remove ‘videos/reco*.png’: No such file or directory


In [203]:
j = 0
c = 0
for i in permuted_indices:
    
    
    #print np.sum(hsvimg-hsvimg2)
    fframe = inframes[i,:,:,:]
    frame = (fframe * 255.0).astype(np.uint8)
    
    bgrimg = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
    
    pframe = model.predict(fframe[np.newaxis,:,:,:], batch_size=1)
    #print mean_squared_error_all
    piframe = (pframe[0,:,:,:]*255.0).astype(np.uint8)
    rgbimg = cv2.cvtColor(piframe, cv2.COLOR_HSV2BGR)
    
    j+=1
    c+=1
    cv2.imwrite("videos/reconstruction1_%05d.png" % (c), rgbimg)
    
    if (j==800):
        for z in range(200):
            pframe = model.predict(pframe, batch_size=1)
            #print mean_squared_error_all
            piframe = (pframe[0,:,:,:]*255.0).astype(np.uint8)
            rgbimg = cv2.cvtColor(piframe, cv2.COLOR_HSV2BGR)
            c+=1
            cv2.imwrite("videos/reconstruction1_%05d.png" % (c), rgbimg)
    if (j % 10==0):
        me = np.mean(np.square((bgrimg.astype(np.float32)/255.0)-(rgbimg.astype(np.float32)/255.0)))
        print "Wrote image %d - MSE: %f" % (j, me)

Wrote image 10 - MSE: 0.059204
Wrote image 20 - MSE: 0.057222
Wrote image 30 - MSE: 0.058343
Wrote image 40 - MSE: 0.059139
Wrote image 50 - MSE: 0.059977
Wrote image 60 - MSE: 0.060064
Wrote image 70 - MSE: 0.058887
Wrote image 80 - MSE: 0.059655
Wrote image 90 - MSE: 0.058536
Wrote image 100 - MSE: 0.055532
Wrote image 110 - MSE: 0.055627
Wrote image 120 - MSE: 0.055849
Wrote image 130 - MSE: 0.054042
Wrote image 140 - MSE: 0.056483
Wrote image 150 - MSE: 0.058199
Wrote image 160 - MSE: 0.059372
Wrote image 170 - MSE: 0.059575
Wrote image 180 - MSE: 0.060520
Wrote image 190 - MSE: 0.060165
Wrote image 200 - MSE: 0.060131
Wrote image 210 - MSE: 0.060248
Wrote image 220 - MSE: 0.060416
Wrote image 230 - MSE: 0.059516
Wrote image 240 - MSE: 0.059927
Wrote image 250 - MSE: 0.059365
Wrote image 260 - MSE: 0.058678
Wrote image 270 - MSE: 0.058924
Wrote image 280 - MSE: 0.059862
Wrote image 290 - MSE: 0.059144
Wrote image 300 - MSE: 0.059225
Wrote image 310 - MSE: 0.059117
Wrote image 320 -

In [208]:
!rm videos/Chimpanzees1_reconstruction5.mp4
!ffmpeg -i videos/reconstruction1_%05d.png -codec:v libx264 -preset slow -b:v 500k -bufsize 1000k -r 10 videos/Chimpanzees1_reconstruction5.mp4
#ffmpeg [input options] -i [input filename] -codec:v [video options] -codec:a [audio options] [output file options] [output filename]
#!rm videos/reco*.png

ffmpeg version N-77455-g4707497 Copyright (c) 2000-2015 the FFmpeg developers
  built with gcc 4.8 (Ubuntu 4.8.4-2ubuntu1~14.04)
  configuration: --extra-libs=-ldl --prefix=/opt/ffmpeg --mandir=/usr/share/man --enable-avresample --disable-debug --enable-nonfree --enable-gpl --enable-version3 --enable-libopencore-amrnb --enable-libopencore-amrwb --disable-decoder=amrnb --disable-decoder=amrwb --enable-libpulse --enable-libdcadec --enable-libfreetype --enable-libx264 --enable-libx265 --enable-libfdk-aac --enable-libvorbis --enable-libmp3lame --enable-libopus --enable-libvpx --enable-libspeex --enable-libass --enable-avisynth --enable-libsoxr --enable-libxvid --enable-libvo-aacenc --enable-libvidstab
  libavutil      55. 11.100 / 55. 11.100
  libavcodec     57. 20.100 / 57. 20.100
  libavformat    57. 20.100 / 57. 20.100
  libavdevice    57.  0.100 / 57.  0.100
  libavfilter     6. 21.101 /  6. 21.101
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  0.100 /  4.  0.100
  libs

In [ ]:
!ls -hs videos/*.mp4

In [209]:
link_video('videos/Chimpanzees1_reconstruction5.mp4')